# I. Loading/Plot functions

In [2]:
DATASETS = {
    'a': 'dogs',
    'b': 'medical-leaf',
    'c': 'texture-dtd',
    'd': 'birds',
    'e': 'AWA',
    'f': 'plt-net',
    'g': 'resisc',
    'h': 'plt-doc',
    'i': 'airplanes',
    '_': 'ALL'
}

color_dict = {
        'erm': 'blue',
        'jtt': 'red',
        'suby': '#00CC96',  # green
        'subg': '#ff7f0e',  # orange
        'rwy': '#00CC96',
        'rwg': '#ff7f0e',
        'dro': '#DEA0FD'  # purple
    }

import wandb
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Initialize wandb
wandb.login()

# Set your entity and project
entity_name = "aureliengauffre"  # e.g., your username or team name
project_name = "SMA_all_2_best"

# Initialize the wandb API ||client
api = wandb.Api()

# Fetch all runs from the specified project
runs = api.runs(f"{entity_name}/{project_name}")

# Create an empty list to hold data for each run
data = []

# Loop through runs and ext/ract the data you're interested in
for run in runs:
    # Extract both summary metrics and config (hyperparameters) for each run
    run_data = {
        "name": run.name,
        "summary_metrics": run.summary._json_dict,
        "config": run.config,
        # Add any other attributes you're interested in here
    }
    data.append(run_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(data)

# For summary metrics and config (hyperparameters), expand them into separate columns
df_summary = pd.json_normalize(df['summary_metrics'])
df_config = pd.json_normalize(df['config'])
df = pd.concat([df.drop(['summary_metrics', 'config'], axis=1), df_summary, df_config], axis=1)
df = df.iloc[:, 1:]  # Drop the first column containing the name of the runs (since there is also another name column)

# Now we have a DataFrame `df` with all runs, their summary metrics, and hyperparameters
# print(df.head())  # Print the first few rows of the DataFrame


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aureliengauffre (use `wandb login --relogin` to force relogin)


In [4]:
df['name']

0            plt-net
1            plt-net
2            plt-net
3            plt-net
4            plt-net
            ...     
3592       airplanes
3593       airplanes
3594    medical-leaf
3595       airplanes
3596       airplanes
Name: name, Length: 3597, dtype: object

In [47]:
def plot_rank(df, x_axis, y_axis, dataset_name, error_bars=None, normalize=False, methods=None):
    print(dataset_name)
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    color_dict = {
        'erm': 'blue',
        'jtt': 'red',
        'suby': '#00CC96',  # green
        'subg': '#ff7f0e',  # orange
        'rwy': '#00CC96',
        'rwg': '#ff7f0e',
        'dro': '#DEA0FD'  # purple
    }
    if dataset_name is None or dataset_name == 'ALL': 
        df_dataset = df
    else:
        df_dataset = df[(df['name'] == dataset_name)]
    
    if normalize:
        df_dataset = normalize_df(df_dataset, metric=y_axis)
    
    fig = go.Figure()  # Initialize a Plotly figure
    all_stats = pd.DataFrame()
    
    # First, calculate means for all methods and combine them
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        stats = df_method.groupby(x_axis)[y_axis].mean().reset_index()
        stats['method'] = method
        all_stats = pd.concat([all_stats, stats], ignore_index=True)
    
    # Now rank the combined data
    all_stats['rank'] = all_stats.groupby(x_axis)[y_axis].rank(ascending=False)
    
    # Plot each method's data
    for method in methods:
        stats = all_stats[all_stats['method'] == method]
        line_style = 'dash' if method in dashed_methods else 'solid'
        color = color_dict.get(method, 'grey')
        fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['rank'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'Rank of {y_axis} vs {x_axis}, dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title='Rank',
                      legend_title='Method',
                      yaxis=dict(autorange='reversed'),  # Reverse the y-axis to show the best rank at the top
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )
    
    # Show the figure
    fig.show()
    return all_stats.pivot_table(index=[x_axis], columns='method', values='rank', aggfunc='first')


dataset_name = 'AWA' #73sports
y_axis = 'mean_grp_acc_te' #'mean_grp_acc_te'
x_axis = 'K'
plot_graph(df, x_axis, y_axis, dataset_name, methods=None)


In [48]:
import plotly.graph_objects as go


def normalize_df(df,metric = 'best_acc_te'):
    # Copy the DataFrame to avoid modifying the original data
    result_df = df.copy()
    
    # Group by 'name', 'K', 'mu', and 'init_seed' and calculate the minimum and maximum accuracy
    grouped = df.groupby(['name', 'K', 'mu', 'init_seed'])[metric]
    min_acc = grouped.transform(np.min)
    max_acc = grouped.transform(np.max)
    
    # Apply the Min-Max normalization formula
    result_df[metric] = (df[metric] - min_acc) / (max_acc - min_acc)
    
    return result_df




def plot_graph(df, x_axis, y_axis, dataset_name, error_bars=None, normalize = False, methods=None,):
    print(dataset_name)
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    color_dict = {
            'erm': 'blue',
            'jtt': 'red',
            'suby': '#00CC96',  # green
            'subg': '#ff7f0e',  # orange
            'rwy': '#00CC96',
            'rwg': '#ff7f0e',
            'dro': '#DEA0FD'  # purple
        }
    if dataset_name is None or dataset_name == 'ALL': 
        df_dataset = df
    else:
        df_dataset = df[(df['name'] == dataset_name)]
    
    if normalize :
        df_dataset = normalize_df(df_dataset,metric=y_axis)
        
    fig = go.Figure()  # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean, standard deviation, and count (for standard error calculation)
        stats = df_method.groupby(x_axis)[y_axis].agg(['mean', 'std', 'count']).reset_index()
        
        # Calculate standard error (SEM)
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        #print(method, stats) # print the number on which we average, interesting !
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color),
                                 error_y=dict(type='data', array=stats['sem'], visible=True)))
        else :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    
    fig.show()


def plot_graph_all(df, x_axis, y_axis, error_bars=None, normalize=False, methods=None):
    """Unlike the original plot_graph function which calculates and plots error bars
    based on individual datasets directly, this version computes the standard error within each dataset first 
    and then averages these errors across all datasets for each method. This approach provides a generalized 
    view of method performance and variability across different datasets."""
    
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if normalize:
        df = normalize_df(df, metric=y_axis)

    fig = go.Figure()  # Initialize a Plotly figure
    all_stats = pd.DataFrame()

    # Process each method separately
    for method in methods:
        df_method = df[df['method'] == method]

        # Group data first by 'name' and then by x_axis and compute statistics
        grouped = df_method.groupby(['name', x_axis])
        stats = grouped[y_axis].agg(['mean', 'std', 'count']).reset_index()

        # Calculate standard error within each dataset
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        
        # Append results to the all_stats DataFrame for later display
        stats['method'] = method
        all_stats = pd.concat([all_stats, stats], ignore_index=True)
        
        # Now group by x_axis and calculate the mean of the means and the mean of the SEMs
        final_stats = stats.groupby(x_axis).agg({'mean': 'mean', 'sem': 'mean'}).reset_index()

        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color),
                                     error_y=dict(type='data', array=final_stats['sem'], visible=True)))
        else:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color)))

    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, Average on all dataset (mean and std)',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

    # Display the all_stats DataFrame
    return all_stats.pivot_table(index=[x_axis, 'name'], columns='method', values=['count', 'sem'], aggfunc='first')



# II. Analysis : K

Here are the **plot parameters** to be played with :

In [49]:
ERROR_BARS = False # Wether to plot the error bars
NORMALIZE = False

To analyse the impact of K, we the **value of mu is fixed**:

In [50]:
df_fix_mu = df[df['name']!='plt-net']
df_fix_mu = df[df['mu']==0.1] # Currently mu in [.05,.1,.2,.4]


df_fix_mu_K2 = df[df['K']==2] 
df_fix_mu_K4 = df[df['K']==4] 
df_fix_mu_K8 = df[df['K']==8] 
df_fix_mu_K12 = df[df['K']==12] 

## a. best_acc_te

In [57]:
plot_rank(df_fix_mu, 'K', 'best_acc_te', 'ALL' , error_bars=False, normalize=False)

ALL


method,dro,erm,jtt,rwg,rwy,subg,suby
K,,,,,,,
2,2.0,5.0,6.0,1.0,4.0,3.0,7.0
4,3.0,1.0,5.0,2.0,4.0,7.0,6.0
8,6.0,1.0,4.0,2.0,3.0,7.0,5.0
12,6.0,1.0,2.0,3.0,4.0,5.0,7.0


In [46]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)


count                                           sem            \
method            dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K  name                                                                         
2  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001904  0.002186   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007826  0.005046   
   birds          5.0  10.0  10.0   5.0   9.0  10.0  10.0  0.002128  0.008549   
   dogs          10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.037444  0.031899   
   medical-leaf  10.0  10.0   9.0  10.0  10.0  10.0  10.0  0.031654  0.022496   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.013855  0.010012   
   plt-net        5.0   5.0  10.0   5.0   5.0  10.0  10.0  0.002490  0.008938   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.012879  0.026115   
   texture-dtd   10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.028123  0.024529   
4  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001634  0.001292   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004582  0.007271   
   birds          5.0   8.0   5.0   5.0   5.0   5.0   5.0  0.000506  0.000906   
   dogs           5.0  10.0  10.0  10.0  10.0  10.0  10.0  0.004559  0.002968   
   medical-leaf   5.0  10.0  10.0   5.0   4.0   5.0  10.0  0.010240  0.015722   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005199  0.005349   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000577  0.001691   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001650  0.005175   
   texture-dtd    8.0  10.0  10.0  10.0  10.0  10.0  10.0  0.018370  0.019022   
8  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001637  0.001730   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006797  0.002115   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000122  0.000223   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003621  0.002795   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007745  0.007370   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006469  0.000546   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002111  0.001878   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005814  0.004459   
   texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.011589  0.011613   
12 AWA            4.0   4.0   5.0   5.0   5.0   5.0   5.0  0.000605  0.000641   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005519  0.000598   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000143  0.000232   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003891  0.002909   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005869  0.005565   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004308  0.002360   
   plt-net        5.0   8.0   2.0   3.0   3.0   NaN   NaN  0.001843  0.000974   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.009661  0.002239   
   texture-dtd   10.0   5.0   5.0  10.0  10.0   4.0  15.0  0.005124  0.003236   

                                                                   
method                jtt       rwg       rwy      subg      suby  
K  name                                                            
2  AWA           0.004352  0.001761  0.005744  0.001585  0.002775  
   airplanes     0.021973  0.017209  0.009027  0.012230  0.010284  
   birds         0.009291  0.003054  0.006870  0.004911  0.010129  
   dogs          0.032296  0.031549  0.027008  0.035920  0.041679  
   medical-leaf  0.024881  0.021323  0.025076  0.040206  0.069037  
   plt-doc       0.036132  0.010792  0.018275  0.003310  0.006143  
   plt-net       0.001831  0.002823  0.002768  0.000593  0.001166  
   resisc        0.035466  0.013755  0.015372  0.013446  0.019709  
   texture-dtd   0.029875  0.029604  0.026636  0.031605  0.031212  
4  AWA           0.001471  0.001201  0.0

In [10]:
NORMALIZE

False

In [11]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [12]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                            sem  \
method            dro   erm   jtt   rwg   rwy  subg  suby        dro   
K  name                                                                
2  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.496147   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0   2.975086   
   birds          5.0  10.0  10.0   5.0  10.0  10.0  10.0   4.406245   
   dogs          10.0  10.0  10.0  10.0  10.0  10.0  10.0   8.357522   
   medical-leaf  10.0  10.0  10.0  10.0  10.0  10.0  10.0  11.662414   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0   7.355354   
   plt-net        5.0   5.0  10.0   5.0   5.0  10.0  10.0   0.892837   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0   6.086303   
   texture-dtd   10.0  10.0  10.0  10.0  10.0  10.0  10.0   6.282665   
4  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.418847   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0   4.838349   
   birds          5.0   9.0   5.0   5.0   5.0   5.0   5.0   0.591491   
   dogs           5.0  10.0  10.0  10.0  10.0  10.0  10.0   3.336587   
   medical-leaf   5.0  10.0  10.0   5.0   5.0   5.0  10.0   4.090600   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.846951   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.895261   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0   2.657296   
   texture-dtd    8.0  10.0  10.0  10.0  10.0  10.0  10.0   4.485180   
8  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.694310   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0   2.576015   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0   0.287098   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.437672   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0   3.468111   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0   2.269098   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.534237   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0   4.455881   
   texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0   0.000000   
12 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0   2.675992   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.361485   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0   0.456863   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0   3.836513   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0   3.488869   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0   1.331721   
   plt-net        5.0   8.0   2.0   3.0   3.0   NaN   NaN   1.974671   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0   3.595159   
   texture-dtd   10.0   5.0   5.0  10.0  10.0   4.0  15.0   0.000000   

                                                                       \
method                erm        jtt        rwg        rwy       subg   
K  name                                                                 
2  AWA           4.150970   1.579483   3.573018   7.045194   0.778211   
   airplanes     8.324796   3.782990   2.966150   8.023533   4.417810   
   birds         2.349546   2.400414   5.995185   2.394584   1.534921   
   dogs          8.205489   6.464903   9.081805   7.728241   9.101726   
   medical-leaf  9.493655  12.945542   9.615765   9.589766   8.897902   
   plt-doc       5.433973   6.193750   9.678324   2.842465  11.424307   
   plt-net       3.921297   0.550992   1.255476   0.702903   0.860606   
   resisc        4.755710   2.497164   5.062870   7.957654   8.894188   
   texture-dtd   2.560985   8.063534   7.002841   5.969171   4.363656   
4  AWA           0.989068   0.993907   1.049754   1.972655   1.577038   
   airplanes     9.235767   2.700165   2.498683   2.858583   1.726951   
   birds         1.333496   0.444122   0.628545   0.746800   1.330835   
   dogs          2.487196   1.489602   5.092102   8.194422   2.679311   
   medical-leaf  9.813155   9.057633  1

In [13]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [14]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                           sem            \
method            dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K  name                                                                         
2  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008360  0.021828   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.028599  0.025387   
   birds          5.0  10.0  10.0   5.0  10.0  10.0  10.0  0.036156  0.015387   
   dogs          10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.059940  0.066093   
   medical-leaf  10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.088853  0.070395   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.073538  0.025593   
   plt-net        5.0   5.0  10.0   5.0   5.0  10.0  10.0  0.009699  0.015554   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.024521  0.034331   
   texture-dtd   10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.088391  0.040441   
4  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001257  0.001423   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.016272  0.013532   
   birds          5.0   9.0   5.0   5.0   5.0   5.0   5.0  0.001791  0.002022   
   dogs           5.0  10.0  10.0  10.0  10.0  10.0  10.0  0.003609  0.005898   
   medical-leaf   5.0  10.0  10.0   5.0   5.0   5.0  10.0  0.011962  0.030509   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.015320  0.015414   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005339  0.005574   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006687  0.009650   
   texture-dtd    8.0  10.0  10.0  10.0  10.0  10.0  10.0  0.024511  0.027844   
8  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001494  0.002146   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003681  0.006826   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000174  0.000474   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007266  0.003168   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008062  0.011863   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006297  0.002407   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006216  0.003260   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008083  0.008160   
   texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.013897  0.011120   
12 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002153  0.001949   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005255  0.002930   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000159  0.007321   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004425  0.003195   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007159  0.009809   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004009  0.002392   
   plt-net        5.0   8.0   2.0   3.0   3.0   NaN   NaN  0.007559  0.003037   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.010478  0.002529   
   texture-dtd   10.0   5.0   5.0  10.0  10.0   4.0  15.0  0.007611  0.003939   

                                                                   
method                jtt       rwg       rwy      subg      suby  
K  name                                                            
2  AWA           0.005897  0.021783  0.032583  0.010731  0.008107  
   airplanes     0.008648  0.031904  0.034839  0.052396  0.027568  
   birds         0.021402  0.027743  0.026921  0.011309  0.038720  
   dogs          0.064816  0.062975  0.055867  0.049538  0.076224  
   medical-leaf  0.076715  0.054270  0.072565  0.069354  0.139048  
   plt-doc       0.065980  0.060733  0.023106  0.071595  0.022725  
   plt-net       0.004410  0.009264  0.008743  0.009713  0.004099  
   resisc        0.005827  0.032789  0.053941  0.035131  0.086189  
   texture-dtd   0.046736  0.044965  0.056025  0.085268  0.069172  
4  AWA           0.001570  0.002046  0.0

In [15]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## d. minor_group_acc

In [16]:
x_axis = 'K'
y_axis = 'minor_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# III. Analysis : mu

Here are the **plot parameters** to be played with :

In [17]:
ERROR_BARS = True #Wether to plot the error bars
NORMALIZE = False

To analyse the impact of mu, we the **value of K is fixed**:

In [18]:
df_fix_K8 = df[df['K']==8] # Currently K in [2,4,8,12]
df_fix_K2 = df[df['K']==2] # Currently K in [2,4,8,12]
df_fix_K4 = df[df['K']==4] # Currently K in [2,4,8,12]

## a. best_acc_te

In [19]:
x_axis = 'mu'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_K8, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                           sem  \
method              dro   erm   jtt   rwg   rwy  subg  suby       dro   
mu   name                                                               
0.05 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002992   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.010417   
     birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000639   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003001   
     medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003857   
     plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002929   
     plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001933   
     resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004824   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.009783   
0.10 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001637   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006797   
     birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000122   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003621   
     medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007745   
     plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006469   
     plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002111   
     resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005814   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.011589   
0.20 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001424   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000475   
     birds          5.0   5.0   5.0   5.0   5.0  10.0  10.0  0.000779   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001264   
     medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.001738   
     plt-doc        5.0   5.0   5.0   5.0   5.0  10.0   NaN  0.002201   
     plt-net        NaN   5.0   5.0   3.0   3.0   5.0   5.0       NaN   
     resisc         8.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005156   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006883   

                                                                               
method                  erm       jtt       rwg       rwy      subg      suby  
mu   name                                                                      
0.05 AWA           0.002323  0.002195  0.003131  0.005197  0.001333  0.005126  
     airplanes     0.002784  0.001780  0.002696  0.003501  0.004118  0.006740  
     birds         0.000648  0.000754  0.001013  0.000500  0.000399  0.000303  
     dogs          0.003588  0.003823  0.003697  0.006104  0.003655  0.006033  
     medical-leaf  0.012974  0.024009  0.007222  0.009421  0.011260  0.014054  
     plt-doc       0.006227  0.004943  0.003293  0.004953  0.004478  0.009267  
     plt-net       0.001055  0.002272  0.000910  0.001800  0.002002  0.002756  
     resisc        0.005187  0.009301  0.002710  0.004139  0.003259  0.006710  
     texture-dtd   0.006837  0.005893  0.005891  0.004668  0.013009  0.006800  
0.10 AWA           0.001730  0.001373  0.001300  0.001499  0.000404  0.001544  
     airplanes     0.002115  0.007500  0.002498  0.001925  0.004342  0.003336  
     birds         0.000223  0.000403  0.000412  0.000206  0.000562  0.000429  
     dogs          0.002795  0.001691  0.002814  0.005555  0.003134  0.003521  
     medical-leaf  0.007370  0.006444  0.006253  0.002610  0.018790  0.011845  
     plt-doc       0.000546  0.006074  0.006382  0.002388  0.004455  0.002523  
     plt-net       0.001878  0.002744  0.002936  0.001092  0.001881  0.002232  
     resisc        0.004459  0.002404  0.001971  0.002048  0.005923  0.001410  
     texture-dtd   0.011613  0.006167  0.003479  0.004818  0.009707  0.005723  
0.20 AWA           0.000955  0.001680  0.001051  0.000725  0.001132  0.001015  
     airplanes     0.000865  0.001167  0.001968  0.000638  0.001062

In [20]:
x_axis = 'mu'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K2, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [21]:
x_axis = 'mu'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_K8, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                           sem  \
method              dro   erm   jtt   rwg   rwy  subg  suby       dro   
mu   name                                                               
0.05 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.913004   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.303030   
     birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.504067   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.677408   
     medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.412508   
     plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.977615   
     plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.323238   
     resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  5.531604   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000000   
0.10 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.694310   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.576015   
     birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.287098   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.437672   
     medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  3.468111   
     plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.269098   
     plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.534237   
     resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  4.455881   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000000   
0.20 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.931990   
     airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.389759   
     birds          5.0   5.0   5.0   5.0   5.0  10.0  10.0  0.871617   
     dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.007084   
     medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.593914   
     plt-doc        5.0   5.0   5.0   5.0   5.0  10.0   NaN  1.859436   
     plt-net        NaN   5.0   5.0   4.0   4.0   5.0   5.0       NaN   
     resisc         8.0   5.0   5.0   5.0   5.0   5.0   5.0  2.025809   
     texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.680414   

                                                                               
method                  erm       jtt       rwg       rwy      subg      suby  
mu   name                                                                      
0.05 AWA           2.827365  2.594125  2.725117  4.078984  3.056242  1.713918  
     airplanes     0.255472  0.220386  0.654234  0.186841  3.855175  0.140469  
     birds         0.500760  0.655119  0.286355  0.829958  0.590586  0.545743  
     dogs          0.596249  0.865555  1.040422  1.376042  1.885632  1.043435  
     medical-leaf  1.474230  0.000000  3.628690  1.462056  2.118302  0.000000  
     plt-doc       2.181480  0.782231  3.206374  1.330285  1.095668  1.996581  
     plt-net       2.817479  2.549606  1.273740  0.832924  0.723811  1.859616  
     resisc        2.552843  0.628138  3.601461  6.044806  2.281743  2.943150  
     texture-dtd   0.000000  0.000000  0.555556  0.000000  2.484520  0.000000  
0.10 AWA           1.393472  1.146975  1.089824  2.177588  0.937472  0.894440  
     airplanes     0.240160  0.539832  1.186174  0.288928  2.407278  0.960639  
     birds         0.380899  0.233373  0.344026  0.400320  0.271782  0.685070  
     dogs          3.115615  1.113209  1.212581  0.875266  3.385686  3.569647  
     medical-leaf  1.743922  4.322629  4.475195  2.628554  4.214666  2.617059  
     plt-doc       2.330608  1.578721  4.006432  2.698443  2.970188  1.374108  
     plt-net       1.291300  1.898871  2.893634  1.178556  0.588794  1.921797  
     resisc        7.913077  1.810777  3.503804  3.124768  0.920909  0.959497  
     texture-dtd   1.111111  1.111111  0.680414  0.555556  3.042903  0.878410  
0.20 AWA           1.976528  2.132808  1.038628  1.616828  0.275682  0.535684  
     airplanes     0.948754  0.336402  1.772654  0.489716  1.351742

In [22]:
x_axis = 'mu'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

NameError: name 'df_fix_K' is not defined

## c. relative_acc

In [ ]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                           sem            \
method            dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K  name                                                                         
2  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008360  0.021828   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.028599  0.025387   
   birds          5.0  10.0  10.0   5.0  10.0  10.0  10.0  0.036156  0.015387   
   dogs          10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.059940  0.066093   
   medical-leaf  10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.088853  0.070395   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.073538  0.025593   
   plt-net        5.0   5.0  10.0   5.0   5.0  10.0  10.0  0.009699  0.015554   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.024521  0.034331   
   texture-dtd   10.0  10.0  10.0  10.0  10.0  10.0  10.0  0.088391  0.040441   
4  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001257  0.001423   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.016272  0.013532   
   birds          5.0   9.0   5.0   5.0   5.0   5.0   5.0  0.001791  0.002022   
   dogs           5.0  10.0  10.0  10.0  10.0  10.0  10.0  0.003609  0.005898   
   medical-leaf   5.0  10.0  10.0   5.0   5.0   5.0  10.0  0.011962  0.030509   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.015320  0.015414   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005339  0.005574   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006687  0.009650   
   texture-dtd    8.0  10.0  10.0  10.0  10.0  10.0  10.0  0.024511  0.027844   
8  AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001494  0.002146   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003681  0.006826   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000174  0.000474   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007266  0.003168   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008062  0.011863   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006297  0.002407   
   plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006216  0.003260   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008083  0.008160   
   texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.013897  0.011120   
12 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002153  0.001949   
   airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005255  0.002930   
   birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000159  0.007321   
   dogs           5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004425  0.003195   
   medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007159  0.009809   
   plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.004009  0.002392   
   plt-net        5.0   8.0   2.0   3.0   3.0   NaN   NaN  0.007559  0.003037   
   resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.010478  0.002529   
   texture-dtd   10.0   5.0   5.0  10.0  10.0   4.0  15.0  0.007611  0.003939   

                                                                   
method                jtt       rwg       rwy      subg      suby  
K  name                                                            
2  AWA           0.005897  0.021783  0.032583  0.010731  0.008107  
   airplanes     0.008648  0.031904  0.034839  0.052396  0.027568  
   birds         0.021402  0.027743  0.026921  0.011309  0.038720  
   dogs          0.064816  0.062975  0.055867  0.049538  0.076224  
   medical-leaf  0.076715  0.054270  0.072565  0.069354  0.139048  
   plt-doc       0.065980  0.060733  0.023106  0.071595  0.022725  
   plt-net       0.004410  0.009264  0.008743  0.009713  0.004099  
   resisc        0.005827  0.032789  0.053941  0.035131  0.086189  
   texture-dtd   0.046736  0.044965  0.056025  0.085268  0.069172  
4  AWA           0.001570  0.002046  0.0

In [ ]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# IV. Examples 

## a. Dataset difficulty :

In [ ]:
def plot_violin(df, metric, category):
    """
    Creates and displays a violin plot for the given DataFrame.
    Parameters:
    - df: pandas DataFrame containing the data to plot.
    - metric: str, the name of the column in df containing the values to plot.
    - category: str, the name of the column in df representing different categories to separate the violins.

    """
    fig = go.Figure()
    categories = df[category].unique()
    for cat in categories:
        cat_data = df[df[category] == cat][metric]
        fig.add_trace(go.Violin(y=cat_data, name=cat, box_visible=True, meanline_visible=True))

    fig.update_layout(title=f"{metric} by {category} for dataset",
                      yaxis_title=metric,
                      legend_title=category)

    # Show the figure
    fig.show()

In [ ]:
plot_violin(df_fix_mu,'best_acc_te','method')

In [ ]:
plot_violin(df_fix_mu_K2,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu_K12,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu,'worst_grp_acc_te','method')

In [ ]:
plot_violin(df_fix_mu,'best_acc_te','name')

In [ ]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [ ]:
plot_violin(df_fix_mu,'worst_grp_acc_te','name')

In [ ]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [ ]:
df_1 = df_fix_mu[df_fix_mu['name'] == 'dogs']
df_1 = df_1[df_1['K'] == 12]
df_1 = df_1[df_1['method'] == 'erm']
df_1['best_acc_te']

2875    0.827342
2878    0.820479
2879    0.813617
2880    0.828649
2881    0.828105
Name: best_acc_te, dtype: float64

In [ ]:
df_2 = df_fix_mu[df_fix_mu['name'] == 'medical-leaf']
df_2 = df_2[df_2['K'] == 12]
df_2 = df_2[df_2['method'] == 'erm']
df_2['best_acc_te']

2547    0.910802
2548    0.883642
2549    0.916049
2550    0.899691
2551    0.904630
Name: best_acc_te, dtype: float64

In [ ]:
df_2['best_acc_te'].std()/(5)**.5 # We check the computation from previous error bar in graphs

0.0055654918002647534